In [59]:
# Import basic libraries
import os
import json
import pandas as pd
import numpy as np
import requests

# Agentic Workflow Related libraries
from dotenv import load_dotenv
from IPython.display import display, Markdown
from openai import OpenAI #llm API handler 

# load env variables
load_dotenv(override = True)
openai = OpenAI()

import gradio as gr

In [6]:
booking_details = pd.read_csv("./data/Booking.csv") # Customer Booking Details
fare_details = pd.read_csv("./data/flight_fares.csv") # Flight Fare Details
flight_info = pd.read_csv("./data/flight_info.csv") # Flight Details

#### PushOver - If the tool call happened

In [73]:
PUSHOVER_USER = os.getenv("PUSHOVER_USER")
PUSHOVER_TOKEN = os.getenv("PUSHOVER_TOKEN")
PUSHOVER_URL = "https://api.pushover.net/1/messages.json"


def push(messages):
    print(f"push : {messages}")
    payload = {"user":PUSHOVER_USER, "token":PUSHOVER_TOKEN, "message":messages}
    requests.post(PUSHOVER_URL, data = payload)

In [74]:
def information_related_to_customer(name:str, mobile_no:str, flight_no:str):
    push(f"Recording customer details {name}, {mobile_no} and {flight_no}")
    return {'recorded':'ok'}

In [75]:
# information_related_to_customer
information_related_to_customer_json = {
    "name":"information_related_to_customer",
    "description": "Use this tool to capture customer information",
    "parameters": {
        "type" : "object",
        "properties" : {
            "name":{
                "type": "string",
                "description": "name of the customer"
            },
            "mobile_no":{
                "type": "number",
                "description": "Mobile No. of the Customer"
            },
            "flight_no":{
                "type": "string",
                "description": "Flight No that the customer booked"
            }
        },
        "required" : ["name", "mobile_no"], 
        "additionalProperties": False
    }
}

In [76]:
tools = [{"type":"function", "function":information_related_to_customer_json}]

In [68]:
# system_prompt = f"You are acting as Pre-flight assistance. Whenever a customer approaches with a question. "
booking_task_llm = "You are 'booking_task_speciality' Pre-flight Assistant, Your main task to book flights for customers, \
    update flight details only. If you encountered question that is out of the topic, please provide message to user, \
    that I can't assist to your request "

router_system_prompt = f"You are acting as Pre-flight assistance. Your main task is to understand the question and direct to relevant department \
Before that first, gather information using the tool information_related_to_customer and record the information and save it repository"

system_prompt = [{'role':'system', 'content': router_system_prompt}]
user_prompt = system_prompt + [{'role':"user", "content":"I want to know about my flights"}]       

In [77]:
router_system_prompt = f"You are acting as Pre-flight assistance. Your main task is to understand the question and direct to relevant department \
Before that first, gather information using the tool information_related_to_customer and record the information and save it repository"

In [ ]:
def handle_tool_calls(tool_calls):
    # :param -> tool_calls : Response generated by LLMs
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name # One of the custom written function we are calling here
        arguments = json.loads(tool_call.function.arguments) # All the arguments that we got from llm
        print(f"Tool called: {tool_name}", flush = True)

        # globals() function gets all the functions that are declared in the python environment
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {} 
        results.append({"role": "tool", "content":json.dumps(result), "tool_call_id": tool_call.id})
    return results

In [79]:
def chat(message, history):
    messages = [{'role':'system', 'content': router_system_prompt}] + \
                history + \
                [{"role":"user", "content": message}]
    done = False
    while not done:
        # Generate the response using gpt-4o-mini llm
        response = openai.chat.completions.create(model = "gpt-4o-mini", 
                                                messages = messages, tools = tools)
        
        # Reason why the model stopped
        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            print(message)
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
        else:
            done = True
            
    return response.choices[0].message.content

In [80]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3F6ZhidHWTaqYIUQGaPwKbwE', function=Function(arguments='{"name": "Sans", "mobile_no": 2339449565, "flight_no": "FY12345"}', name='information_related_to_customer'), type='function')])
Tool called: information_related_to_customer
push : Recording customer details Sans, 2339449565 and FY12345


Traceback (most recent call last):
  File "/Volumes/Transcend/DeepLearning/.venv_agentic_ai/lib/python3.13/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/Volumes/Transcend/DeepLearning/.venv_agentic_ai/lib/python3.13/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/Volumes/Transcend/DeepLearning/.venv_agentic_ai/lib/python3.13/site-packages/gradio/blocks.py", line 2147, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/Volumes/Transcend/DeepLearning/.venv_agentic_ai/lib/python3.13/site-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await fn(*processed_input)
       